In [8]:
import pandas as pd
import ast
import gurobipy as gp
from gurobipy import GRB, quicksum

In [151]:
B = 200
M = 10000
T_start = 480
T_end = 1260

In [129]:
# data load
df = pd.read_excel('./Dataset_v4.xlsx')
# df['travel_times'] = df['travel_times'].apply(lambda x: ast.literal_eval(x))

In [135]:
DP = df.loc[df['DP']=='DP2','Name'].item()
print(DP)

10 Bayfront Ave, Singapore 018956


In [136]:
# df = df[~df.DP.isin(['DP2','DP3'])].reset_index(drop=True) 
df = df[~df.DP.isin(['DP1','DP3'])].reset_index(drop=True) 

# distance 

In [137]:
import pandas as pd

# Load the CSV data into a pandas DataFrame
df_distance = pd.read_excel('./distance_in_min_v4.xlsx', index_col='x')

# Initialize an empty dictionary for the travel times
t = {}

# Iterate through each row of the DataFrame
for index, row in df_distance.iterrows():
    t[index] = row.to_dict()

# Print or use the travel_times dictionary
print(t)

{'60 Bukit Timah Rd, B1-02, Singapore 229900': {'60 Bukit Timah Rd, B1-02, Singapore 229900': 1, '300 Tiong Bahru Rd, Singapore 168731': 14, '26 Bussorah St, Singapore 199444': 7, '261 Victoria St, Singapore 189876': 7, '10, Telok Blangah Green, 109178': 19, 'Gardens by the Bay': 10, 'Wings of Time': 21, 'Night Safari Singapore': 28, 'Marina Bay Sands Skypark Observation Deck': 12, 'Singapore Cable Car': 19, 'Jewel Changi Airport': 21, 'S.E.A Aquarium': 21, 'Singapore Zoo': 27, 'Singapore River Cruise': 12, 'ArtScience Museum at Marina Bay Sands': 10, 'Singapore Flyer + Time Capsule': 10, '1 Imbiah Rd, 099692': 22, '40 Imbiah Rd, Sentosa, 099700': 22, 'Adventure Cove Waterpark': 21, 'National Gallery Singapore': 11, 'River Wonders': 27, 'National Museum of Singapore': 6, 'Science Centre Singapore': 26, 'National Orchid Garden': 18, 'Asian Civilisations Museum': 11, '80 Siloso Road, Southside, Blk B, #01-04, 098969': 24, 'SkyHelix Sentosa': 22, '21 Jurong Town Hall Rd, Singapore 609433'

In [145]:
A = df[df.type!='start_end']['Name'].tolist()   # Activities
F_b = df[df.type=='Fb']['Name'].tolist() # Breakfast activities
F_l = df[df.type=='Fl']['Name'].tolist()     # Lunch activities
F_d = df[df.type=='Fd']['Name'].tolist()    # Dinner activities
T_set = df[df.type=='tours']['Name'].tolist()    # Total of 4 tour activities
R = df[df.type=='attractions']['Name'].tolist()  # Total of 3 attraction activities

# Nodes
N = df['Name'].tolist()

# Time parameters (s = earliest start time, e = latest end time)
s = dict()
for k, v in df[df.type!='start_end'][['Name','Start']].values:
    s.update({k:v})
    
e = dict()
for k, v in df[df.type!='start_end'][['Name','End']].values:
    e.update({k:v})

# Duration parameter
d = dict()
for k, v in df[df.type!='start_end'][['Name','Duration']].values:
    d.update({k:v})

# cost parameter
c = dict()
for k, v in df[df.type!='start_end'][['Name','Price']].values:
    c.update({k:v})

# Travel times (in minutes)
# t = dict()
# for k, v in df[df.type!='start_end'][['Name','travel_times']].values:
#     t.update({k:v})

In [146]:
# rating parameter
u = dict()
for k, v in df[df.type!='start_end'][['Name','Rating']].values:
    u.update({k:v})

In [152]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import GRB, quicksum

# Create a new model
m = gp.Model("TripScheduling")

In [153]:
# Activity Selection Variables (x_i)
x = m.addVars(A, vtype=GRB.BINARY, name="x")
# Sequencing Variables (y_{ij})
y = m.addVars(N, N, vtype=GRB.BINARY, name="y")
# Start Time Variables (T_i)
T = m.addVars(A, vtype=GRB.CONTINUOUS, name="T")
# Sequence Position Variables (tau_i)
tau = m.addVars(A, vtype=GRB.CONTINUOUS, lb=1, name="tau")

In [154]:
#Trip start and ends at specific designated point (home)
m.addConstr(gp.quicksum(y[DP, j] for j in A) == 1, name="StartAtDP") # Start of the trip
m.addConstr(gp.quicksum(y[i, DP] for i in A) == 1, name="EndAtDP") # End of the trip

#Trip duration from T_start to T_end
total_duration = gp.quicksum(d[i] * x[i] for i in A) + gp.quicksum(t[i][j] * y[i, j] for i in N for j in N if i != j)
m.addConstr(total_duration <= T_end - T_start, "TotalDuration") # inequality -- can allow some extra time (for feasibility)

#Flow conservation for activities
for j in A:
    # Inbound flow equals activity selection
    m.addConstr(gp.quicksum(y[i, j] for i in N if i != j) == x[j], name=f"InFlow_{j}")
    # Outbound flow equals activity selection
    m.addConstr(gp.quicksum(y[j, k] for k in N if k != j) == x[j], name=f"OutFlow_{j}")

#Time window
for i in A:
    m.addConstr(T[i] >= s[i], name=f"StartTimeLB_{i}")
    m.addConstr(T[i] <= e[i] - d[i], name=f"StartTimeUB_{i}")

#Sequencing and Time
for i in N:
    for j in N:
        if i != j:
            if i == DP:
                # From DP to first activity
                m.addConstr(
                    T[j] >= T_start + t[i][j] - M * (1 - y[i, j]),
                    name=f"TimeFromDP_{j}"
                )
            elif j == DP:
                # From last activity to DP (no T['DP'] needed)
                pass
            else:
                # Between activities
                m.addConstr(
                    T[j] >= T[i] + d[i] + t[i][j] - M * (1 - y[i, j]),
                    name=f"TimeSeq_{i}_{j}"
                )

#Budget constraint
m.addConstr(gp.quicksum(c[i] * x[i] for i in A) <= B, name="BudgetConstraint")

#Food activity constraint
m.addConstr(gp.quicksum(x[i] for i in F_b) == 1, name="BreakfastConstraint")
m.addConstr(gp.quicksum(x[i] for i in F_l) == 1, name="LunchConstraint")
m.addConstr(gp.quicksum(x[i] for i in F_d) == 1, name="DinnerConstraint")

#Tour and attraction constraint
m.addConstr(gp.quicksum(x[i] for i in T_set) >= 1, name="TourConstraint")
m.addConstr(gp.quicksum(x[i] for i in R) >= 1, name="AttractionConstraint")

#Subtour eliminatin constraint
for i in A:
    for j in A:
        if i != j:
            m.addConstr(
                tau[i] - tau[j] + len(A) * y[i, j] <= len(A) - 1,
                name=f"SubtourElim_{i}_{j}"
            )

#Domain of sequence variable
for i in A:
    m.addConstr(tau[i] >= 1, name=f"TauLB_{i}")

In [155]:
import gurobipy as gp
from gurobipy import GRB

# 1st Round: Maximize Enjoyment

# Define the objective function to maximize enjoyment
m.setObjective(gp.quicksum(u[i] * x[i] for i in A), GRB.MAXIMIZE)

# Optimize the model for the 1st round
m.optimize()

# Check if the optimization was successful
if m.status == GRB.OPTIMAL:
    # Store the maximum enjoyment score from the 1st round
    max_enjoyment = m.ObjVal
    print(f"1st Round Optimization Successful: Max Enjoyment = {max_enjoyment}")
else:
    print("1st Round Optimization Failed")
    exit()

if m.status == GRB.INFEASIBLE:
    print("Model is infeasible. Running infeasibility analysis...")
    m.computeIIS()
    m.write("infeasible_constraints.ilp")
    
# 2nd Round: Minimize Total Travel Time

# Update the objective function to minimize total travel time
m.setObjective(gp.quicksum(t[i][j] * y[i, j] for i in N for j in N if i != j), GRB.MINIMIZE)

# Add the additional constraint for minimum enjoyment (at least 90% of the maximum enjoyment from the 1st round)
m.addConstr(gp.quicksum(u[i] * x[i] for i in A) >= 0.90 * max_enjoyment, "MinEnjoymentConstraint")

# Optimize the model for the 2nd round
m.optimize()

# Check if the 2nd round optimization was successful
if m.status == GRB.OPTIMAL:
    total_travel_time = m.ObjVal
    print(f"2nd Round Optimization Successful: Minimized Total Travel Time = {total_travel_time} minutes")

    # Extract the optimal solution
    selected_activities = [i for i in A if x[i].X > 0.5]
    # Get the start times of activities
    activity_start_times = {i: T[i].X for i in selected_activities}

    # Build the sequence of the trip
    sequence = []
    current_node = DP
    visited = set()
    while True:
        for j in N:
            if y[current_node, j].X > 0.5 and j != current_node and (current_node, j) not in visited:
                sequence.append((current_node, j))
                visited.add((current_node, j))
                current_node = j
                break
        else:
            break
        if current_node == DP:
            break

    # Build the itinerary
    itinerary = []
    time_pointer = T_start  # Start time of the trip
    for arc in sequence:
        i, j = arc
        if j != DP:
            # Travel from i to j
            travel_time = t[i][j]
            arrival_time = time_pointer + travel_time
            itinerary.append({
                'start_time': time_pointer,
                'end_time': arrival_time,
                'description': f"Travel from {i} to {j}"
            })

            # Activity at j
            activity_start = max(arrival_time, s[j])
            idle_time = activity_start - arrival_time
            if idle_time > 0:
                itinerary.append({
                    'start_time': arrival_time,
                    'end_time': activity_start,
                    'description': f"Wait until {j} opens"
                })
            activity_end = activity_start + d[j]
            activity_type = (
                'Breakfast' if j in F_b else
                'Lunch' if j in F_l else
                'Dinner' if j in F_d else
                'Tour' if j in T_set else
                'Attraction' if j in R else
                'Activity'
            )
            itinerary.append({
                'start_time': activity_start,
                'end_time': activity_end,
                'description': f"{activity_type} at {j}"
            })
            time_pointer = activity_end
        else:
            # Travel back to DP
            travel_time = t[i][j]
            itinerary.append({
                'start_time': time_pointer,
                'end_time': time_pointer + travel_time,
                'description': "Travel back home"
            })
            time_pointer += travel_time

    # Print the itinerary
    print("\nOptimal Schedule:")
    for segment in itinerary:
        start_time = segment['start_time']
        end_time = segment['end_time']
        description = segment['description']
        start_hour = int(start_time // 60)
        start_minute = int(start_time % 60)
        end_hour = int(end_time // 60)
        end_minute = int(end_time % 60)
        start_period = 'AM' if start_hour < 12 or start_hour == 24 else 'PM'
        end_period = 'AM' if end_hour < 12 or end_hour == 24 else 'PM'
        start_hour_formatted = start_hour if start_hour <= 12 else start_hour - 12
        if start_hour_formatted == 0:
            start_hour_formatted = 12
        end_hour_formatted = end_hour if end_hour <= 12 else end_hour - 12
        if end_hour_formatted == 0:
            end_hour_formatted = 12
        print(f"## {start_hour_formatted:02d}:{start_minute:02d}{start_period} ~ {end_hour_formatted:02d}:{end_minute:02d}{end_period} : {description}")

    # Print total enjoyment and travel time
    print(f"\nTotal Enjoyment: {max_enjoyment}")
    print(f"Total Travel Time: {total_travel_time} minutes")

else:
    print("2nd Round Optimization Failed")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 36999 rows, 18901 columns and 164970 nonzeros
Model fingerprint: 0xa1a1b85c
Variable types: 270 continuous, 18631 integer (18631 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 15939 rows and 10971 columns
Presolve time: 0.29s
Presolved: 21060 rows, 7930 columns, 76113 nonzeros
Variable types: 203 continuous, 7727 integer (7727 binary)

Root relaxation: objective 4.381593e+01, 968 iterations, 0.04 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   43.81593    0   17          -   43.81593      -     -    0s
H    

In [ ]:
# Optimal Schedule:
# ## 08:00AM ~ 08:10AM : Travel from 10 Bayfront Ave, Singapore 018956 to Singapore Zam Zam Restaurant
# ## 08:10AM ~ 09:10AM : Breakfast at Singapore Zam Zam Restaurant
# ## 09:10AM ~ 09:12AM : Travel from Singapore Zam Zam Restaurant to 26 Bussorah St, Singapore 199444
# ## 09:12AM ~ 10:00AM : Wait until 26 Bussorah St, Singapore 199444 opens
# ## 10:00AM ~ 12:00PM : Tour at 26 Bussorah St, Singapore 199444
# ## 12:00PM ~ 12:07PM : Travel from 26 Bussorah St, Singapore 199444 to Sufood
# ## 12:07PM ~ 01:07PM : Lunch at Sufood
# ## 01:07PM ~ 01:11PM : Travel from Sufood to Singapore River Cruise
# ## 01:11PM ~ 02:11PM : Attraction at Singapore River Cruise
# ## 02:11PM ~ 02:31PM : Travel from Singapore River Cruise to 40 Imbiah Rd, Sentosa, 099700
# ## 02:31PM ~ 04:01PM : Attraction at 40 Imbiah Rd, Sentosa, 099700
# ## 04:01PM ~ 04:02PM : Travel from 40 Imbiah Rd, Sentosa, 099700 to SkyHelix Sentosa
# ## 04:02PM ~ 04:32PM : Attraction at SkyHelix Sentosa
# ## 04:32PM ~ 04:33PM : Travel from SkyHelix Sentosa to 1 Imbiah Rd, 099692
# ## 04:33PM ~ 05:33PM : Attraction at 1 Imbiah Rd, 099692
# ## 05:33PM ~ 05:50PM : Travel from 1 Imbiah Rd, 099692 to 26 Beach Rd, B1-22 South Beach Avenue, Singapore 189768
# ## 05:50PM ~ 06:00PM : Wait until 26 Beach Rd, B1-22 South Beach Avenue, Singapore 189768 opens
# ## 06:00PM ~ 08:00PM : Dinner at 26 Beach Rd, B1-22 South Beach Avenue, Singapore 189768
# ## 08:00PM ~ 08:05PM : Travel from 26 Beach Rd, B1-22 South Beach Avenue, Singapore 189768 to Singapore Flyer + Time Capsule
# ## 08:05PM ~ 09:05PM : Attraction at Singapore Flyer + Time Capsule
# ## 09:05PM ~ 09:10PM : Travel back home

# Total Enjoyment: 42.00000000000001
# Total Travel Time: 72.0 minutes